In [1]:
import json
import networkx as nx
import numpy as np
import plotly.graph_objects as go

In [2]:
filename = "example-2.json"

In [3]:
with open(filename) as f:
    data = json.load(f)

In [4]:
graph = nx.Graph(directed=True)
graph.add_nodes_from(map(lambda scene: scene['scene_id'], data['scenes']))
graph.add_edges_from(
    (choice['next_scene'], scene['scene_id'])
    for scene in data['scenes']
    for choice in scene['choices']
)

In [6]:
# Генерируем координаты вершин графа
pos = nx.spring_layout(graph, k=0.5, iterations=50)

coords = np.array(list(pos.values()))


# Извлекаем информацию о положении вершин и рёбер графа
node_x = coords[:, 0] # Заполняем списки вершин
node_y = coords[:, 1]
node_text = list(graph.nodes())
edge_x, edge_y = [], []

for edge in graph.edges():
  n_1 = node_text.index(edge[0])
  n_2 = node_text.index(edge[1])
  edge_x.extend([node_x[n_1], node_x[n_2], None])
  edge_y.extend([node_y[n_1], node_y[n_2], None])

print(edge_x, edge_y)

[np.float64(0.3333491558730948), np.float64(-0.09625590464193248), None, np.float64(0.3333491558730948), np.float64(0.7154514410058098), None, np.float64(-0.09625590464193248), np.float64(-0.5371667074100004), None, np.float64(-0.09625590464193248), np.float64(-0.1652038634590499), None, np.float64(0.7154514410058098), np.float64(0.8099489002006074), None, np.float64(0.7154514410058098), np.float64(0.9231804693080453), None, np.float64(-0.5371667074100004), np.float64(-0.598595938551501), None, np.float64(-0.5371667074100004), np.float64(-0.7893707060241294), None, np.float64(-0.1652038634590499), np.float64(-0.06268536845019157), None, np.float64(-0.1652038634590499), np.float64(-0.31249321662937096), None, np.float64(0.8099489002006074), np.float64(0.5851825560578976), None, np.float64(0.8099489002006074), np.float64(0.7895271435585727), None, np.float64(0.9231804693080453), np.float64(0.7988435472215233), None, np.float64(0.9231804693080453), np.float64(0.9646517393184015), None, np

In [7]:
# Визуализируем рёбра графа
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Визуализируем вершины графа
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        color='lightblue',
        size=10,
        line_width=2
    )
)

# Визуализируем общий результат
fig = go.Figure(
 data=[edge_trace, node_trace],
 layout=go.Layout(
  title='Scene Graph',
  showlegend=False,
  hovermode='closest',
  margin=dict(b=20, l=5, r=5, t=40),
  xaxis=dict(showgrid=False, zeroline=False),
  yaxis=dict(showgrid=False, zeroline=False),
  height=1000, width=1000
 )
)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()